# Flask with Config

### Introduction

Now to setup flask with tests, we need to change some of the ways that we setup the application.  The complicated becomes fairly complicated, but luckily we do not need to fully understand all of it, as the setup is the same from application to application.  Still, let's spend some time walking through it.

### Enabling Different Configuration

The main difference that occurs when we run tests is that our configuration of our Flask application is depending on whether we are running tests, working with the application in development mode on our computer, or in production mode when our application is deployed.  

> By configuration we mean, for example, the database that the application is connected to (test or development) but also if the application is in debug mode, or testing mode, which makes it easier to read error messages in the application.  

Let's stake a look at the way that we can configure a flask application.

In [1]:
from flask import Flask
app = Flask(__name__)

In [2]:
app.config

<Config {'ENV': 'production', 'DEBUG': False, 'TESTING': False, 'PROPAGATE_EXCEPTIONS': None, 'PRESERVE_CONTEXT_ON_EXCEPTION': None, 'SECRET_KEY': None, 'PERMANENT_SESSION_LIFETIME': datetime.timedelta(days=31), 'USE_X_SENDFILE': False, 'SERVER_NAME': None, 'APPLICATION_ROOT': '/', 'SESSION_COOKIE_NAME': 'session', 'SESSION_COOKIE_DOMAIN': None, 'SESSION_COOKIE_PATH': None, 'SESSION_COOKIE_HTTPONLY': True, 'SESSION_COOKIE_SECURE': False, 'SESSION_COOKIE_SAMESITE': None, 'SESSION_REFRESH_EACH_REQUEST': True, 'MAX_CONTENT_LENGTH': None, 'SEND_FILE_MAX_AGE_DEFAULT': datetime.timedelta(seconds=43200), 'TRAP_BAD_REQUEST_ERRORS': None, 'TRAP_HTTP_EXCEPTIONS': False, 'EXPLAIN_TEMPLATE_LOADING': False, 'PREFERRED_URL_SCHEME': 'http', 'JSON_AS_ASCII': True, 'JSON_SORT_KEYS': True, 'JSONIFY_PRETTYPRINT_REGULAR': False, 'JSONIFY_MIMETYPE': 'application/json', 'TEMPLATES_AUTO_RELOAD': None, 'MAX_COOKIE_SIZE': 4093}>

We can see tha tconfig is essentially a dictionary that allows us to specify different options about the environment, and behavior of the application.  For us, we'll mainly work with `DEBUG`, `TESTING` and `DATABASE` keys.

For example, when we are about to run our tests, we can set this configuration with the following:

In [3]:
app.config.from_mapping(
        DATABASE='skuawk_test',
        DEBUG = True,
        TESTING = True
    )

True

And now when we look at those specific configuration values, we can see that they are set. 

In [6]:
app.config["DATABASE"], app.config["DEBUG"], app.config["TESTING"]

('skuawk_test', True, True)

In development, we'll want our application to be created with a different configuration -- where the database connects to development and testing is set to false.

To ease creation of our flask app, we can write a function that will, create, configure and return our flask app.  Here it is:

In [8]:
def create_app(database='squawk_development', testing = False, debug = True):
    """Create and configure an instance of the Flask application."""
    app = Flask(__name__)
    app.config.from_mapping(
        DATABASE=database,
        DEBUG = debug,
        TESTING = testing
    )
    return app

So now we can create an app for development just by calling `create_app`, and can create an application for our tests, by calling `create_app` with the following arguments.

In [9]:
app = create_app(database='squawk_test', testing = True)

In [10]:
app.config['DATABASE']

'squawk_test'

Now there is still one problem that we have, and that is that our app knows absolutely nothing about any routes that we set up.

In [14]:
app.url_map

Map([<Rule '/static/<filename>' (GET, OPTIONS, HEAD) -> static>])

We want our create_app function to return a flask app that is setup and ready to go, so we actually define our routes inside the `create_app` function.  We can do so with the following:

In [15]:
def create_app(database='squawk_development', testing = False, debug = True):
    """Create and configure an instance of the Flask application."""
    app = Flask(__name__)
    app.config.from_mapping(
        DATABASE=database,
        DEBUG = debug,
        TESTING = testing
    )
    
    @app.route('/')
    def root_url():
        return 'Welcome to squawk'

    @app.route('/restaurants')
    def restaurants():
        conn = get_db()
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM restaurants;')
        records = cursor.fetchall()
        return json.dumps(records, default = str)

    @app.route('/restaurants/<id>')
    def restaurant(id):
        conn = get_db()
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM restaurants WHERE id = %s;', (id,))
        records = cursor.fetchall()
        return json.dumps(records, default = str)

    return app

So now when we create our application, our routes should be attached.

In [16]:
app = create_app(database='squawk_test', testing = True)

In [17]:
app.url_map

Map([<Rule '/restaurants' (GET, OPTIONS, HEAD) -> restaurants>,
 <Rule '/' (GET, OPTIONS, HEAD) -> root_url>,
 <Rule '/restaurants/<id>' (GET, OPTIONS, HEAD) -> restaurant>,
 <Rule '/static/<filename>' (GET, OPTIONS, HEAD) -> static>])

Nice how that works.

If you look through the application, you'll notice that we moved the `create_app` function to the `src/__init__` file, and then if we want to run the application, we call the `run.py` file.  

That file simply creates a new flask application, and then runs `app.run`.

```python
from src import create_app

app = create_app()
app.run(debug = True)
```

### Connecting to the Database

The other new function that we have is the `get_db` function, defined in the `db.py` file.  Remember that every time we hit a new route, we made a new connection to the database.  

When we call the `get_db` function, we initialize a new connection to the database with the following: 

```python

from flask import current_app

#  The real function is slightly more complicated
def get_db():
    db = psycopg2.connect(user = 'postgres', password = 'postgres',
            dbname = current_app.config['DATABASE'])
    return db
```

Notice that this function, connects to the database by looking at the `current_app`'s configuration.  So when we initialize our app with our test database, this will connect to `squawk_test`, and when we initialize our app with our development database (as in the `run.py` file), this will connect to the `squawk_development`.

So now that we have our `get_db` function, we call `get_db` in each of the routes (look in the `create_app` function above), and then get the cursor from there.

> Also note that we have a `close_db` function, that closes the connection to the database.  In the tests we are careful to close connections between database calls, to avoid weird errors where our tests hang, and do not complete.

### Final Changes

Now there is one last change in our routes that we have there, and that is a switch from jsonify to `json.dumps(records, default = str)`.  This is because of some limitations with jsonify.  It has issues converting timestamps, and type decimals, which we get back from our database to json.  If we use the `json.dumps(records, default = str)`, these issues are avoided.

### Summary

In this lesson, we saw how we can get our flask application to play nicely with our tests.  Doing so meant that we needed some easy ways to create instances of our flask app with different configurations.  To that end, we defined a `create_app` function that allowed for us to create an app with different configuration.  In that function, we also declare our routes for our application.  

We also defined a `get_db` function to ease the retrieval of our database.  This function looks to the database specified in the configuration of our flask app, and returns a connection to that database.

Finally, we moved over to the `json.dumps(records, default = str)` function, coming from the simplejson library, so that we can convert datetimes and decimals to json.

### Resources

[Flask Config](https://flask.palletsprojects.com/en/1.1.x/config/)